# IBM Ponder This - January 2025

## Problem Statement

Three jugs A, B, and C have volumes $V_A=\sqrt{5}$, $V_B=\sqrt{3}$, and $V_C=\sqrt{2}$ liters, respectively. There is a tap from which we can fill the jugs, and a sink into which we can empty them. At the beginning of the problem, the three jugs are empty.

To achieve the solution of the problem, we perform a sequence of steps that can only be one of the following:

Pour water from the tap into a jug until it is filled to the top.
Empty a jug into the sink.
Pour water from one jug into another until either the source jug is empty or the receiving jug is filled to the top, whichever happens first.
We assume that these steps are performed with absolute accuracy and no water is spilled.

Our goal is to measure a specific amount of water, meaning we seek to arrive at a state where any one of the jugs (no matter which one) contains this specific amount. As it is impossible to measure with these jugs any rational, non-zero volume with a finite number of steps, we will relax the condition: Instead of measuring a volume of $1$, we measure a volume of $1 \pm 0.0003$ liters, i.e., any volume in the interval $[0.9997, 1.0003]$ will suffice.

The solution should be presented as a list of two-letter strings that represent the steps of the solution, in which the first letter denotes the source, and the second letter denotes the recipient. The tap is denoted by T and the sink by S. The list should also be preceded by a number representing the minimum number of steps.

For example: If the task is to measure $1 \pm 0.01$ liters, the fastest solution would require six steps and could be described as

6 TA AB BS AB TA AB

Upon completion of the above sequence of six steps, jug A would contain 1.00803 liters of water, a volume within the specified tolerance.

Your goal: Find a sequence of steps measuring $1 \pm 0.0003$ liters such that the number of steps is minimal. Submit your answer using the above format.

A bonus "*" will be given for solving the following problem: Keeping the same jugs $A$ and $B$, replace jug $C$ with one of the volume $V_C$ such that:

- $V_C$ is a rational number: $V_C = \frac{p}{q}$, where p and q are integers.
- $V_C < V_B$.
- $1\pm 10^{-8}$ liters can be measured with 11 steps.
- 11 is the minimum possible number of steps.
- $q$ has the smallest possible value.
- Submit $V_C$ in the form $\frac{p}{q}$ and a description of the steps in the above format.

## Solution

The base problem can be solved with a BFS (Breadth First Search) approach. We initialise a double-ended queue with the state where each jug has no water in it. At each step, we pop the state at the bottom of the queue, compute all the potential next states based on the possible actions and add those states to the top of the queue. Along the way, we keep track of the path that has been followed. As soon as we reach the target amount of water, we have the solution. 

In order to avoid issues related to floating point numbers, we can verify that we find the same answer using sympy for the irrational numbers or by widening the margin around the target (which in this case keeps giving the same answer even for a relatively large widening).

In [1]:
import math
from collections import deque

def measure_water(x: int, y: int, z: int, target: int, delta: float):

    queue = deque([(0, 0, 0, [], [])])  # (a, b, c, steps)
    visited = set()

    while queue:
        a, b, c, steps, res = queue.popleft()

        # Check if exactly one jug has the target amount
        if target - delta <= a <= target + delta or target - delta <= b <= target + delta or target - delta <= c <= target + delta:
            return str(len(steps)) + " " + " ".join(steps)

        if (a, b, c) in visited:
            continue
        visited.add((a, b, c))

        # Generate all possible next states
        next_states = [
            (x, b, c, steps + ["TA"], res + [(x, b, c)]),
            (a, y, c, steps + ["TB"], res + [(a, y, c)]),
            (a, b, z, steps + ["TZ"], res + [(a, b, z)]),
            (0, b, c, steps + ["AS"], res + [(0, b, c)]),
            (a, 0, c, steps + ["BS"], res + [(a, 0, c)]),
            (a, b, 0, steps + ["CS"], res + [(a, b, 0)]),
            (a - min(a, y - b), b + min(a, y - b), c, steps + ["AB"], res + [(a - min(a, y - b), b + min(a, y - b), c)]),
            (a - min(a, z - c), b, c + min(a, z - c), steps + ["AC"], res + [(a - min(a, z - c), b, c + min(a, z - c))]),
            (a + min(b, x - a), b - min(b, x - a), c, steps + ["BA"], res + [(a + min(b, x - a), b - min(b, x - a), c)]),
            (a, b - min(b, z - c), c + min(b, z - c), steps + ["BC"], res + [(a, b - min(b, z - c), c + min(b, z - c))]),
            (a + min(c, x - a), b, c - min(c, x - a), steps + ["CA"], res + [(a + min(c, x - a), b, c - min(c, x - a))]),
            (a, b + min(c, y - b), c - min(c, y - b), steps + ["CB"], res + [(a, b + min(c, y - b), c - min(c, y - b))]),
        ]

        # Add valid next states to the queue
        for next_state in next_states:
            if 0 <= next_state[0] <= x and 0 <= next_state[1] <= y and 0 <= next_state[2] <= z:
                queue.append(next_state)

    return []  # If we exhaust the queue, the target is not reachable.


measure_water(math.sqrt(5), math.sqrt(3), math.sqrt(2), 1, 0.0003)

'39 TZ CA TZ CA AS CA TZ CA TZ AB CA TZ CA AS CA TZ CA AS CA TZ CA TZ CA AS CA TZ CA TZ CA AS CA TZ CA AS CA TZ CA TZ CA'